# New South Wales Department of Education (NSW DOE) - Data Case Study 
## Data Analysis

In [ ]:
import pandas as pd
import duckdb
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns

### connect to the database

In [ ]:
con = duckdb.connect('../../database/nsw_doe_data_case_study.duckdb',read_only=False)

#### Exploratory Data Analysis:

In [ ]:
df = con.sql('select * from multi_age_composite_unp').df()

In [ ]:
print(df.shape)
df.head()

In [ ]:
df = con.sql('select * from public_school_nsw_master_dataset').df()
profile_public_school_nsw_master_dataset = ProfileReport(df, title="Public School NSW Data Profiling Report")
profile_public_school_nsw_master_dataset.to_file("profile_public_school_nsw_master_dataset.html")
profile_public_school_nsw_master_dataset

In [ ]:
df = con.sql('select * from multi_age_composite_unpivoted').df()
profile_multi_age_composite_unpivoted = ProfileReport(df, title="Multi Age Composite Profiling Report")
profile_multi_age_composite_unpivoted.to_file("profile_multi_age_composite_unpivoted.html")
profile_multi_age_composite_unpivoted

In [ ]:
df = con.sql('select * from student_attendance_dataset').df()
profile_student_attendance_dataset = ProfileReport(df, title="Student Attendance Profiling Report")
profile_student_attendance_dataset.to_file("profile_student_attendance_dataset.html")
profile_student_attendance_dataset

In [ ]:
# Retrieving the data from views into pandas DataFrames
school_master_data = pd.read_sql("SELECT * FROM school_master_data", con)
school_attendance_data = pd.read_sql("SELECT * FROM school_attendance_data", con)
school_composite_data = pd.read_sql("SELECT * FROM school_composite_data", con)
school_characteristics = pd.read_sql("SELECT * FROM school_characteristics", con)
print("Attendance Data Summary:")
print(school_attendance_data['Attendance_pct'].describe())
print("\nComposite Class Counts Summary:")
print(school_composite_data['Composite_Value'].describe())
print("\nCharacteristics Summary:")
print(school_characteristics[['latest_year_enrolment_FTE', 'Indigenous_pct', 'LBOTE_pct']].describe())

In [ ]:
# Histogram for Attendance Rates
sns.histplot(school_attendance_data['Attendance_pct'], kde=True)
plt.title("Distribution of Attendance Rates")
plt.show()
# Histogram for Composite Class Counts
sns.histplot(school_composite_data['Composite_Value'], kde=True)
plt.title("Distribution of Composite Class Counts")
plt.show()


In [ ]:
#merged_data = pd.merge(school_attendance_data, school_composite_data, on=['School_Code', 'Year'], how='inner')
# Rename the column in school_composite_data before merging
merged_data = pd.merge(school_attendance_data, 
                       school_composite_data.rename(columns={"School_code": "School_Code"}), 
                       on=['School_Code', 'Year'], 
                       how='inner')

# Average attendance rates over years
average_attendance_per_year = merged_data.groupby('Year')['Attendance_pct'].mean()
average_attendance_per_year.plot()
plt.title("Average Attendance Rate Over Years")
plt.ylabel("Attendance Rate")
plt.show()

# Average composite class counts over years
average_composite_per_year = merged_data.groupby('Year')['Composite_Value'].mean()
average_composite_per_year.plot()
plt.title("Average Composite Class Counts Over Years")
plt.ylabel("Composite Counts")
plt.show()


In [ ]:
# Define constants
years = ['14', '15', '16', '17', '18', '19', '21', '22']
measures = ['COMPOSITE_CLASS_COUNT', 'COMPOSITE_CLASS_STUDENTS', 'PCT_COMPOSITE_CLASSES', 'PCT_COMPOSITE_CLASS_STUDENTS']
# Flatten the table
flattened_queries = []
for year in years:
    year_cols = [f"{measure}_{year} AS {measure}" for measure in measures]
    query = f"""
    SELECT 
        School_code,
        School_Name,
        '20{year}' AS Year,
        {', '.join(year_cols)}
    FROM multi_age_composite_dataset
    """
    flattened_queries.append(query)
final_flattened_query = "\nUNION ALL\n".join(flattened_queries)
con.execute(f"CREATE TABLE multi_age_composite_unp AS {final_flattened_query}")


### Action: Do your analysis below

In [ ]:
print(merged_data.dtypes)